In [5]:

"""
input for matrix, for which we want to calculate the Jordan nomal form 
"""

#A = Matrix(QQ,[[2, 1, 2],[1,2, 2], [1, 1, 3]])
#A = Matrix(QQ,[[4, 0, 0],[0,4, -1], [0, 1, 2]])
#A = Matrix(QQ,[[2, 1, -1, 8, -3],[0, 2 , 0, 7, 5], [0, 0, 2,7,5 ],[0, 0, 0, 2, 0], [0, 0, 0, 0, 2]])
#A = Matrix(QQ,[[1, 1, 1, -1, 0],[0, 1, 0, 0, 1],[0, 0 , 0, 1, 0], [0, 0, -1,2,1 ],[0, 0, 0, 0, 1]])
A = Matrix(QQ, [[1, 1, 6, -2],[0, 1, -3, 2],[0, 0, 1, 0],[0, 0, -2, 2]])

"""
The transformation matrix we will be calculating later
"""
X = []

In [6]:
"""
Check if we can determine the Jordan normal form of a given matrix.

A: matrix we want to check
"""
def matrix_is_valid (A):
    return A.ncols() == A.nrows()

print(matrix_is_valid(A))

True


In [7]:


"""
Get array of eigenvalues and the quantities in which they arise.

A: matrix we want to get the eigenvalues of
"""
def get_eigenvalues(A)->[float]:
    #create the characteristic polynomial and return its roots 
    #sort the array, by the eigenvalues
    poly = A.charpoly() 
    return sorted(poly.roots(ring=QQ, multiplicities=True))

val = get_eigenvalues(A)
print(val)

[(1, 3), (2, 1)]


In [8]:
"""
insert the eigenvalues allong the diagonal of a identity matrix 

arr: array with the eigenvalues
"""
def construct_matrix_with_eigenvalues(arr):
    #calculate the size of the matrix
    matrix_size = 0
    for i in range(len(arr)):       
        matrix_size = matrix_size + arr[i][1]
    
    #create the matrix
    M = identity_matrix(matrix_size)
    
    #insert values
    n = 0
    for i in range(len(arr)): 
        for j in range(arr[i][1]):
            M[n,n] = arr[i][0]
            n = n+1
    return M

M = construct_matrix_with_eigenvalues(val)
print(M)

[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 2]


In [9]:

"""
Check if a new vector is linearly dependent on a set of vectors.

v: the new vector
S: array of already existing vectors
"""

def check_if_vektor_is_lineary_dependant_and_add(v,S): 
    rank_prior = Matrix(S).rank()
    S.append(v)
    rank_new = Matrix(S).rank()
    if rank_prior == rank_new:
        S = S[:-1]   
    return S

S = [(0, 1, 0), (0, 0, 1)]
v1 = (0, 1, 0)
print(check_if_vektor_is_lineary_dependant_and_add(v1, S))

S = [(0, 1, 0), (0, 0, 1)]
v2 = (1, 0, 0)
print(check_if_vektor_is_lineary_dependant_and_add(v2, S))

[(0, 1, 0), (0, 0, 1)]
[(0, 1, 0), (0, 0, 1), (1, 0, 0)]


In [10]:
"""
One of the basis vectors has to be linearly independent of all existing vectors.
Check all basis vectors until a linearly independent vector is found and return it.

unfiltered_existing_vectors: linearly independent vectors (array also contains 0 vectors, these are to be ignored in calculations)

span: the vector space in which all the vectors are


"""

span = [(0, 1, 0 , 0, 0),(0, 0, 0 , 1, 0)]
existing = [(0, 0, 0 , 1, 0),(0, 0 , 0 , 0, 0), (0, 0 , 0 , 0, 0)]
def lin_independant_vect(unfiltered_existing_vectors,span):
    #filter existing vectors (stored in unfiltered_existing_vectors)
    filtered_temp_arr = [element for element in unfiltered_existing_vectors if element != zero_vector(QQ, len(span[0]))]
    
    existing_vectors = Matrix(filtered_temp_arr)
    
    #get next lineary indipendant vector in vectorspace
    existing_matrix = Matrix(filtered_temp_arr)

    for x in span:
        rank_old = existing_matrix.rank()
        new_matrix = existing_matrix.stack(Matrix([x]))
        rank_new = new_matrix.rank()
        if rank_old != rank_new:
            return x
    
print(lin_independant_vect(existing, span))

(0, 1, 0, 0, 0)


In [11]:
"""
Calculate the top vector of column i in the matrix Z (here called temp_array).

i: index of column
span_nr: rank of newly calculated eigenvector
kernel_span: array with all eigenspaces of all powers
temp_arr: array that holds information on all already calculated vectors
"""
def calc_generalized_eigenvector(i,span_nr, kernel_span, temp_arr):
    if i == 0:
        #get the first vector of the span, since no vector have been calculated yet, we can take any vector
        return kernel_span[span_nr][-1]
    else: 
        #get the vectors that make up the span in which the new vector must be 
        span_end = len(kernel_span[span_nr])
        if span_nr-1 >= 0:
            span_start = len(kernel_span[span_nr-1])
        else:
            span_start = 0
        span = kernel_span[span_nr][span_start:span_end]
        
        #calculate the new vector using gramm schmidt
        return lin_independant_vect(temp_arr[span_nr], span)

In [12]:
"""
Calculate the transformation matrix such that A = X * M * X^-1 (this function is iterated over for each eigenvalue).

kernel_span: all spans of eigenspaces of all powers for the given eigenvalue
Z: matrix used to determine the number of Jordan blocks for the given eigenvalue
M = A - aI, where a is the eigenvalue
"""
def caclulate_tranformation_Matrix(kernel_span, Z, M):
    temp_arr =[[zero_vector(QQ, M.ncols()) for _ in range(Z.ncols())] for _ in range(Z.ncols())]
    for i in range(Z.ncols()):
        #array to store the calculated vectors of a clomn of Z
        x_temp_array = []
        #get the power of the Eigenvectorspace we are in
        span_nr = Z.column(i).list().count(1)-1
        if span_nr < 0: break
        #calculate the vector of the Eigenvectorspace with the highest power for defined column in Z
        a = calc_generalized_eigenvector(i, span_nr, kernel_span, temp_arr)
        
        #save the vector
        x_temp_array.append(a)
        temp_arr[span_nr][i] = a
        
        #calculate vectors withhin eigenvectorspaces of lesser power
        for j in range(span_nr):
            a = M*a
            temp_arr[span_nr-j-1][i] = a
            x_temp_array.append(a)
        
        
        #revesre the order of the vectors and add them to the transformationmatrix
        X.extend(reversed(x_temp_array))
    

In [13]:
"""
Construct Z to help determine the size of Jordan blocks.

dim_diff_array: array with the differences between the dimensions of the eigenspaces
"""
dim_diff = [3, 2]
def construct_Z(dim_diff_array):
    size = max(dim_diff_array[0], len(dim_diff_array))
    Z = zero_matrix(QQ, size)
    for j in range(len(dim_diff_array)):
        for k in range(dim_diff_array[j]):  
            Z[j, k] = 1
    return Z
print(construct_Z(dim_diff))

[1 1 1]
[1 1 0]
[0 0 0]


In [14]:
"""
Calculate the sequence along the upper diagonal and create the corresponding vectors of the transformation matrix.

arr: array with eigenvalues
A: the matrix we want to calculate it for

"""        

def calc_sequence(arr,A):
    sequence = []
    for i in range(len(arr)):
        #array with spans of all eigenvectorspaces of all powers
        kernel_span = []
        #array with the span of the current eigenvectorspace 
        S = []
        #array used to construct Z 
        dim_diff_array = []
        
        M = A- arr[i][0]*identity_matrix(A.rank())
        dim_old = 0
        B = A- arr[i][0]*identity_matrix(A.rank())
        
        #get the span of all eigenvector spaces of all powers and saves them
        #calculates dim_diff to find out the size of the jordanblocks
        while True:
            C = B.transpose().kernel()
        
            dim_diff = C.dimension()-dim_old
            if dim_diff == 0: break
            dim_diff_array.append(C.dimension()-dim_old)
            for x in C.basis():
                S= check_if_vektor_is_lineary_dependant_and_add(x, S)
            kernel_span.append(S.copy())
            dim_old = B.kernel().dimension()
            B = B*M
        
        #create Z to find out size of jordan blocks
        
        Z = construct_Z(dim_diff_array)
        
        #calculate sequence for this eigenvalue
        for x in Z.columns():
            for j in range(list(x).count(1) - 1):
                sequence.append(1)
            sequence.append(0)
        sequence = sequence[:-1]        
        
        caclulate_tranformation_Matrix(kernel_span, Z, M)
        
    #cuts off the last element, becuase sequence has to fit into upper diagonal
    if len(sequence) == rank(A):sequence = sequence[:-1]
    return sequence
seq = calc_sequence(val,A)

print(seq)



[1, 1, 0]


In [15]:
"""
insert the sequence into the upper diagonal of the matrix

M: matrix with the eigenvalues on the diagoal
seq: the sequence
"""
def insert_sequence(M, seq):
    for i in range(len(seq)):
        M[i, i+1] = seq[i]
    return M


In [16]:
X = []
def calc_jordan_normal_form(A):
    if matrix_is_valid(A): 
        eigenvalues = get_eigenvalues(A)
        M = construct_matrix_with_eigenvalues(eigenvalues)
        seq = calc_sequence(eigenvalues, A)
        M = insert_sequence(M, seq)
        return M

J = calc_jordan_normal_form(A)
"""
print out the results
"""
X_1 = Matrix(X).transpose()
print("the Jordan-Normal-Form of A ist:")
print(J)
print()

print("X")
print(X_1)

X_2 = Matrix(X).transpose().inverse()
print("X^-1")
print(X_2)
print("X_1 * J * X_2A")
print(X_1 * J * X_2)
print("J is the jordan nomalform: ",X_1 * J * X_2 == A)

        



the Jordan-Normal-Form of A ist:
[1 1 0 0]
[0 1 1 0]
[0 0 1 0]
[0 0 0 2]

X
[  1   2   0   0]
[  0   1   0   1]
[  0   0   1   0]
[  0   0   2 1/2]
X^-1
[ 1 -2 -8  4]
[ 0  1  4 -2]
[ 0  0  1  0]
[ 0  0 -4  2]
X_1 * J * X_2A
[ 1  1  6 -2]
[ 0  1 -3  2]
[ 0  0  1  0]
[ 0  0 -2  2]
J is the jordan nomalform:  True


'\n        B = A- arr[i][0]*identity_matrix(A.rank())\n        C = B.transpose().kernel()\n        dim_diff_array.append(C.dimension())\n        for x in C.basis():\n            S.append(x)\n        kernel_span.append(S.copy())\n'